##  NoSQL Database :: MongoDB

In [4]:
import pandas as pd

In [2]:
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27017")
db = client.apan5400

#### Read CSV File

In [5]:
df = pd.read_csv("companies.csv")

#### Convert DataFrame to dictionary format

In [7]:
records = df.to_dict("records")

#### Insert records into MongoDB

In [8]:
db.companies.insert_many(records)

InsertManyResult([ObjectId('65f65c8537df8ee976fb44ec'), ObjectId('65f65c8537df8ee976fb44ed'), ObjectId('65f65c8537df8ee976fb44ee'), ObjectId('65f65c8537df8ee976fb44ef'), ObjectId('65f65c8537df8ee976fb44f0'), ObjectId('65f65c8537df8ee976fb44f1'), ObjectId('65f65c8537df8ee976fb44f2'), ObjectId('65f65c8537df8ee976fb44f3'), ObjectId('65f65c8537df8ee976fb44f4'), ObjectId('65f65c8537df8ee976fb44f5'), ObjectId('65f65c8537df8ee976fb44f6'), ObjectId('65f65c8537df8ee976fb44f7'), ObjectId('65f65c8537df8ee976fb44f8'), ObjectId('65f65c8537df8ee976fb44f9'), ObjectId('65f65c8537df8ee976fb44fa')], acknowledged=True)

## 1) Companies Established in the Last 25 Years in Descending Order of Revenue

In [27]:
recent_companies = db.companies.find({"founded": {"$gte": 1999}}).sort("revenue", -1) # cutoff year (2024 - 25 = 1999)

for company in recent_companies:
    print(f"{company['company']}, Revenue: {company['revenue']} billion USD")

Alibaba Group, Revenue: 134.57 billion USD
Tesla, Revenue: 81.46 billion USD


## 2) Total of Annual Revenues of Companies Headquartered in Asia

In [38]:
asian_company_revenue = db.companies.aggregate([
    {
        "$match": {
            "country": {"$in": ["China", "South Korea", "Japan", "Saudi Arabia"]}
        }
    },
    {
        "$group": {
            "_id": None,
            "total_revenue": {"$sum": "$revenue"}
        }
    }
])

for result in asian_company_revenue:
    print(f"Total Revenue: {result['total_revenue']} billion USD")

Total Revenue: 1699.54 billion USD


## 3) Country of the Leading Car and Truck Manufacturer

In [36]:
leading_car = db.companies.find_one({
    "description": {
        "$regex": "leading car and truck manufacturer",
        "$options": "i"  # Case-insensitive search
    }
})

print(leading_car["country"])

Germany


## 4) Company Established After 1970 with the Smallest Number of Employees

In [30]:
smallest_company = db.companies.find({"founded": {"$gt": 1970}}).sort("employees", 1).limit(1)

for company in smallest_company:
    print(f"{company['company']}, Number of Employees: {company['employees']}")

Tesla, Number of Employees: 127855


## 5) Annual Revenues and Founding Years of Automotive Companies

In [31]:
automotive_companies = db.companies.find({"description": {"$regex": "automotive", "$options": "i"}}, {"company": 1, "revenue": 1, "founded": 1})

for company in automotive_companies:
    print(f"{company['company']}, Revenue: {company['revenue']} billion USD, Founded: {company['founded']}")

Mercedes-Benz Group, Revenue: 153.95 billion USD, Founded: 1926
Toyota Motor, Revenue: 267.02 billion USD, Founded: 1937
